In [ ]:
# 2025.09.18 - train for xBD & mwBTFreddy

%cd /storage/alperengenc/change_detection/ChangeMamba_AG
%ls

import os
# get_ipython().system = os.system

from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


cfg_dict = {
    "MambaBDA_Tiny" : '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_tiny_224_0229flex.yaml',
    "MambaBDA_Small": '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_small_224.yaml',
    "MambaBDA_Base" : '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/configs/vssm1/vssm_base_224.yaml',
}
pretrained_dict = {
    "MambaBDA_Tiny" : '/storage/alperengenc/change_detection/ChangeMamba_AG/pretrained_weight/vssm_tiny_0230_ckpt_epoch_262.pth',
    "MambaBDA_Small": '/storage/alperengenc/change_detection/ChangeMamba_AG/pretrained_weight/vssm_small_0229_ckpt_epoch_222.pth',
    "MambaBDA_Base" : '/storage/alperengenc/change_detection/ChangeMamba_AG/pretrained_weight/vssm_base_0229_ckpt_epoch_237.pth',
}

MODEL_NAME = 'MambaBDA_Base'
DATASET = "xBD" # "xBD" || "mwBTFreddy"

cfg_file = cfg_dict[MODEL_NAME]
pretrained_file = pretrained_dict[MODEL_NAME]

logfilename = f"/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/train_{now}_{DATASET}_{MODEL_NAME}.log"
pidfilename = logfilename.replace(".log", ".pid.log")
print(logfilename)
print(pidfilename)


"""
Paper parameters:
- AdamW optimizer //train_MambaBDA.py line 89
- learning rate 1e-4
- weight decay 5e-3
- batch size 16
- # training iterations 50000 //800_000 in github repo example
- training data augmentation: random rotation, left-right & top-bottom flip
"""

COMMAND = f"""
nohup python -u changedetection/script/train_MambaBDA.py \
     --dataset                {DATASET} \
     --batch_size             4 \
     --crop_size              256 \
     --max_iters              400_000 \
     --model_type             {MODEL_NAME} \
     --model_param_path       '/storage/alperengenc/change_detection/ChangeMamba_AG/changedetection/deneme_saved_models/tr{now}' \
     --train_dataset_path     '/storage/alperengenc/change_detection/xBD_complete_png/train_combined' \
     --train_data_list_path   '/storage/alperengenc/change_detection/xBD_complete_png/train_combined_list.txt' \
     --test_dataset_path      '/storage/alperengenc/change_detection/xBD_complete_png/test' \
     --test_data_list_path    '/storage/alperengenc/change_detection/xBD_complete_png/test_list.txt' \
     --cfg                    {cfg_file} \
     --logfile                {logfilename} \
     --pretrained_weight_path {pretrained_file} \
     --learning_rate          0.0001 \
     --weight_decay           0.005 \
     >/dev/null 2>&1 & echo $! > {pidfilename}
""".replace("\n", "")
#*-- last line alternative for realtime stdout along with file (tee)
# 2>&1 | tee -a {logfilename} & echo $! > {pidfilename}
# >> {logfilename} 2>&1 & echo $! > {pidfilename}

os.system(command=COMMAND)

# BEFORE RERUNNING: check if still running:
# ps -fp $(cat <pidfilename>) || echo "not running"
!ps -fp $(cat {pidfilename}) || echo "not running" && rm {pidfilename}


/storage/alperengenc/change_detection/ChangeMamba_AG
analyze/          kernels/      pipeline_changemamba.ipynb  requirements.txt
changedetection/  LICENSE       pretrained_weight/
classification/   LOGLAR_CMAG/  README.md
figures/          modelcard.sh  README_zh-CN.md
/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/train_2025-09-19_18-02-44_xBD_MambaBDA_Base.log
/storage/alperengenc/change_detection/ChangeMamba_AG/LOGLAR_CMAG/train_2025-09-19_18-02-44_xBD_MambaBDA_Base.log
UID          PID    PPID  C STIME TTY          TIME CMD
alperen+ 1566682       1  0 18:02 ?        00:00:01 python -u changedetection/sc
